In [2]:
import os
import cv2
import shutil
import random
import numpy as np
import tensorflow as tf
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed

In [33]:
print(tf.__version__)
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
img_to_array = tf.keras.preprocessing.image.img_to_array
array_to_img = tf.keras.preprocessing.image.array_to_img

2.18.0


**raw video renamed and moved to project videos**

In [14]:
# Source and destination paths
source_folder = r'D:\PycharmProjects\pro_dis_2\collected_data\!test\set_70'
destination_folder = r'D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_70'

# List of words for renaming
words = ["cup", "eat", "go", "hot", "sit"]

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
    print(f"Created destination folder: {destination_folder}")

# Copy all video files from the source folder to the destination folder
video_files = [f for f in os.listdir(source_folder) if f.endswith('.mp4')]
for file in video_files:
    shutil.copy2(os.path.join(source_folder, file), destination_folder)
    print(f"Copied {file} to {destination_folder}")

# Get all video files in the copied folder
video_files = [f for f in os.listdir(destination_folder) if f.endswith('.mp4')]

# Create a list of tuples (file_name, date_modified)
video_files_with_date = [(file, os.path.getmtime(os.path.join(destination_folder, file))) for file in video_files]

# Sort files by date modified (earliest first)
video_files_with_date.sort(key=lambda x: x[1])

# Rename each file based on the words list
for i, (file, _) in enumerate(video_files_with_date):
    if i < len(words):
        new_name = f"{words[i]}.mp4"
        old_path = os.path.join(destination_folder, file)
        new_path = os.path.join(destination_folder, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed {file} to {new_name}")
    else:
        print("Not enough words in the list to rename all video files.")
        break

print("\nAll videos are renamed and saved in project videos folder.")

Created destination folder: D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_70
Copied VID_20241119_091928.mp4 to D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_70
Copied VID_20241119_091933.mp4 to D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_70
Copied VID_20241119_091939.mp4 to D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_70
Copied VID_20241119_091944.mp4 to D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_70
Copied VID_20241119_091949.mp4 to D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_70
Renamed VID_20241119_091928.mp4 to cup.mp4
Renamed VID_20241119_091933.mp4 to eat.mp4
Renamed VID_20241119_091939.mp4 to go.mp4
Renamed VID_20241119_091944.mp4 to hot.mp4
Renamed VID_20241119_091949.mp4 to sit.mp4

All videos are renamed and saved in project videos folder.


**frame extraction and save in set_01 and 01.png**

In [15]:
def extract_frame(frame, output_dir, frame_index):
    """
    Save a single frame as a .png image in the specified directory.
    Ensures complete image save before moving to the next frame.
    """
    frame_filename = os.path.join(output_dir, f"{frame_index:02d}.png")  # Format frame index with leading zeros
    cv2.imwrite(frame_filename, frame)
    return frame_filename

def extract_frames_with_priority_deletion(video_path, output_dir, target_frames=60):
    """
    Extract frames from the video, handling cases with more or fewer frames than the target.
    If the video has more frames, remove 20% from the start and 80% from the end.
    If the video has fewer frames, copy the last frame to fill the deficit.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    os.makedirs(output_dir, exist_ok=True)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    # Handle case when the video has fewer than the target number of frames
    if total_frames < target_frames:
        # Save all existing frames
        for i in range(total_frames):
            extract_frame(frames[i], output_dir, i + 1)  # Start frame index from 1

        # Copy the last frame to fill the deficit until the target number is reached
        last_frame = frames[-1]
        for i in range(total_frames, target_frames):
            extract_frame(last_frame, output_dir, i + 1)

        print(f"Copied last frame to fill the deficit for {video_path}.")
        cap.release()
        return

    # Handle case when the video has more than the target number of frames
    if total_frames > target_frames:
        frames_to_delete = total_frames - target_frames
        delete_from_end = int(frames_to_delete * 0.8)  # 80% of frames to delete from the end
        delete_from_start = frames_to_delete - delete_from_end  # 20% from the start

        # Retain the middle portion after deleting the required frames
        frames = frames[delete_from_start:total_frames - delete_from_end]

    # Extract frames after deletion logic or for target-sized videos
    extracted_frame_count = 0

    with ThreadPoolExecutor() as executor:
        futures = []
        for i in range(len(frames)):
            future = executor.submit(extract_frame, frames[i], output_dir, extracted_frame_count + 1)
            futures.append(future)
            extracted_frame_count += 1

        for future in as_completed(futures):
            _ = future.result()

    cap.release()
    print(f"Extracted {extracted_frame_count} frames from {video_path}")

def process_multiple_video_folders(base_video_folder, output_base_folder):
    for i in range(61, 71):  # Loop to cover set_01 to set_50
        set_name = f"set_{i:02d}"  # Format set number with leading zeros (e.g., set_01, set_02)
        video_folder = os.path.join(base_video_folder, set_name)

        if not os.path.exists(video_folder):
            print(f"Skipping {video_folder} as it does not exist.")
            continue

        output_set_folder = os.path.join(output_base_folder, set_name)

        for video_file in os.listdir(video_folder):
            if video_file.endswith('.mp4'):
                video_path = os.path.join(video_folder, video_file)
                word_name = os.path.splitext(video_file)[0]
                word_output_dir = os.path.join(output_set_folder, word_name)
                print(f"Processing {video_file} in {set_name}...")
                extract_frames_with_priority_deletion(video_path, word_output_dir, target_frames=60)

# Base folder paths
base_video_folder = r'D:\PycharmProjects\pro_dis_2\collected_data\!test\videos'
output_base_folder = r'D:\PycharmProjects\pro_dis_2\collected_data\!test\extracted frames'

# Call the function
process_multiple_video_folders(base_video_folder, output_base_folder)

print("\nAll frames are extracted")

Processing cup.mp4 in set_61...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_61\cup.mp4
Processing eat.mp4 in set_61...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_61\eat.mp4
Processing go.mp4 in set_61...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_61\go.mp4
Processing hot.mp4 in set_61...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_61\hot.mp4
Processing sit.mp4 in set_61...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_61\sit.mp4
Processing cup.mp4 in set_62...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_62\cup.mp4
Processing eat.mp4 in set_62...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data\!test\videos\set_62\eat.mp4
Processing go.mp4 in set_62...
Extracted 60 frames from D:\PycharmProjects\pro_dis_2\collected_data

**original augmentation before cropping**

In [213]:
# Define source and output directories
source_dir = r'D:\PycharmProjects\pro_dis_2\collected_data\!source_27_11_night\1_extracted_frames_10w'
output_base_dir = r'D:\PycharmProjects\pro_dis_2\collected_data\2_new_augmented_datasets'

# ImageDataGenerator for basic augmentations
datagen = ImageDataGenerator(
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    # rotation_range=5,
    # zoom_range=[0.8, 1.2],
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # shear_range=0.2
)

# Function to apply noise to an image
def add_noise(image):
    row, col, ch = image.shape
    mean = 0
    sigma = 0.1
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy_image = np.clip(image + gauss * 255, 0, 255).astype(np.uint8)
    return noisy_image

# Function to downsample (simulate lower resolution)
def downsample_image(image, scale=0.5):
    h, w = image.shape[:2]
    image = cv2.resize(image, (int(w * scale), int(h * scale)))
    return cv2.resize(image, (w, h))

# Function to apply blur
def apply_blur(image, kernel_size=3):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

# Function to augment a single frame
def augment_frame(image_path, output_dir, frame_idx):
    img = cv2.imread(image_path)
    img = img_to_array(img)

    # Apply basic augmentations
    img = datagen.random_transform(img)

    # Additional augmentations
    if random.random() < 0.3:
        img = add_noise(img)
    if random.random() < 0.2:
        img = downsample_image(img)
    if random.random() < 0.2:
        img = apply_blur(img)

    # Convert to image format and save
    aug_img = array_to_img(img)
    aug_img.save(f"{output_dir}/{frame_idx:02d}.png")

# Function to process each frame folder
def process_frame_folder(source_word_dir, output_word_dir):
    frames = sorted(os.listdir(source_word_dir))
    num_frames = len(frames)

    # Process frames in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(augment_frame, os.path.join(source_word_dir, frame), output_word_dir, idx + 1)
            for idx, frame in enumerate(frames)
        ]

        # Ensure all frames are processed
        for future in as_completed(futures):
            future.result()

# Main loop to generate augmented datasets
set_folders = sorted(os.listdir(source_dir))

for set_folder in set_folders:
    set_path = os.path.join(source_dir, set_folder)
    if os.path.isdir(set_path):
        output_set_dir = os.path.join(output_base_dir, set_folder)
        os.makedirs(output_set_dir, exist_ok=True)

        # Process each word folder in the set
        word_folders = os.listdir(set_path)
        for word_folder in word_folders:
            source_word_dir = os.path.join(set_path, word_folder)
            output_word_dir = os.path.join(output_set_dir, word_folder)
            os.makedirs(output_word_dir, exist_ok=True)

            if os.path.isdir(source_word_dir):
                print(f"Processing {set_folder}/{word_folder}...")
                process_frame_folder(source_word_dir, output_word_dir)

print("\nAugmented datasets created successfully.")

# Validation step to check for 60 frames in each word folder
all_ok = True
for set_folder in os.listdir(output_base_dir):
    set_dir = os.path.join(output_base_dir, set_folder)
    for word_folder in os.listdir(set_dir):
        word_folder_path = os.path.join(set_dir, word_folder)
        if os.path.isdir(word_folder_path):
            frame_count = len(os.listdir(word_folder_path))
            if frame_count != 60:
                print(f"Warning: {set_folder}/{word_folder} has {frame_count} frames instead of 60.")
                all_ok = False

if all_ok:
    print("All sets and word folders have exactly 60 frames. Validation successful!")

Processing set_01/bat...
Processing set_01/cup...
Processing set_01/drop...
Processing set_01/eat...
Processing set_01/fish...
Processing set_01/hot...
Processing set_01/jump...
Processing set_01/milk...
Processing set_01/pen...
Processing set_01/red...
Processing set_02/bat...
Processing set_02/cup...
Processing set_02/drop...
Processing set_02/eat...
Processing set_02/fish...
Processing set_02/hot...
Processing set_02/jump...
Processing set_02/milk...
Processing set_02/pen...
Processing set_02/red...
Processing set_03/bat...
Processing set_03/cup...
Processing set_03/drop...
Processing set_03/eat...
Processing set_03/fish...
Processing set_03/hot...
Processing set_03/jump...
Processing set_03/milk...
Processing set_03/pen...
Processing set_03/red...
Processing set_04/bat...
Processing set_04/cup...
Processing set_04/drop...
Processing set_04/eat...
Processing set_04/fish...
Processing set_04/hot...
Processing set_04/jump...
Processing set_04/milk...
Processing set_04/pen...
Processin

**run crop_lip.py**

**3rd try augmentation after cropping - used in final - it is a different augmentation. Shouldn't apply any other before this.**

In [ ]:
import os
import cv2
import numpy as np
import random
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from albumentations import Compose, HorizontalFlip, RandomBrightnessContrast, Rotate, Affine, GaussianBlur, GaussNoise, RGBShift, OpticalDistortion, Perspective, Equalize

# Source and destination directories
source_dir = r"E:\1_cropped_frames"
destination_dir = r"E:\2_augmented_frames"
os.makedirs(destination_dir, exist_ok=True)

# Target size for resizing images
TARGET_SIZE = (112, 80)  # (Width, Height)

# Albumentations augmentations
def get_augmentations(augmentation_type):
    """Return augmentation pipeline based on the specified type."""
    augmentations = {
        "HorizontalFlip": Compose([HorizontalFlip(p=1.0)]),
        "Brightness": Compose([RandomBrightnessContrast(
            brightness_limit=(0.3, 0.3), contrast_limit=0.0, p=1.0)]),
        "Contrast": Compose([RandomBrightnessContrast(
            brightness_limit=0.0, contrast_limit=(0.3, 0.3), p=1.0)]),
        "Rotate": Compose([Rotate(limit=5, p=1.0)]),
        "Translate": Compose([Affine(translate_px={"x": (-10, 10), "y": (-10, 10)}, p=1.0)]),
        "Scale": Compose([Affine(scale=(0.9, 1.1), p=1.0)]),
        "GaussianBlur": Compose([GaussianBlur(blur_limit=(7, 15), p=1.0)]),
        "GaussNoise": Compose([GaussNoise(var_limit=(100.0, 250.0), p=1.0)]),
        "RGBShift": Compose([RGBShift(r_shift_limit=50, g_shift_limit=50, b_shift_limit=50, p=1.0)]),
        "OpticalDistortion": Compose([OpticalDistortion(distort_limit=0.2, shift_limit=0.2, p=1.0)]),
        "Perspective": Compose([Perspective(scale=(0.05, 0.1), p=1.0)]),
        "Equalize": Compose([Equalize(p=1.0)])
    }
    return augmentations.get(augmentation_type)

# Custom augmentations
def downsample_image(image, scale=0.5):
    """Downsample and restore the image to simulate lower resolution."""
    h, w = image.shape[:2]
    image = cv2.resize(image, (int(w * scale), int(h * scale)))
    return cv2.resize(image, (w, h))

def sharpen_image(image):
    """Apply sharpening to the image."""
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def temporal_shift(frames):
    """Shift frames temporally by a random offset."""
    shift_amount = random.randint(-2, 2)
    return np.roll(frames, shift_amount, axis=0)

def process_cropped_image(cropped_image):
    """Process cropped image with LAB adjustment, CLAHE, blurring, and sharpening."""
    lab_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(3, 3))
    l_channel_eq = clahe.apply(l_channel)
    lab_image_eq = cv2.merge((l_channel_eq, a_channel, b_channel))
    bgr_image_eq = cv2.cvtColor(lab_image_eq, cv2.COLOR_LAB2BGR)
    blurred_image = cv2.GaussianBlur(bgr_image_eq, (7, 7), 0)
    edge_preserved_image = cv2.bilateralFilter(blurred_image, d=5, sigmaColor=75, sigmaSpace=75)
    sharpening_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened_image = cv2.filter2D(edge_preserved_image, -1, sharpening_kernel)
    final_image = cv2.GaussianBlur(sharpened_image, (5, 5), 0)
    return final_image

# Resizing function
def resize_with_interpolation(image, target_size):
    """Resize image with appropriate interpolation based on size."""
    original_h, original_w = image.shape[:2]
    target_w, target_h = target_size

    # Check if resizing is necessary
    if (original_h, original_w) == (target_h, target_w):
        return image  # No resizing needed

    # Determine interpolation method
    if original_h > target_h or original_w > target_w:  # Downscaling
        interpolation = cv2.INTER_AREA
    else:  # Upscaling
        interpolation = cv2.INTER_LINEAR

    return cv2.resize(image, (target_w, target_h), interpolation=interpolation)

# Function to augment an image
def augment_image(image, augmentation_type):
    """Apply augmentation based on the type."""
    if augmentation_type in [
        "HorizontalFlip", "Brightness", "Contrast", "Rotate", "Translate", "Scale",
        "GaussianBlur", "GaussNoise", "RGBShift", "OpticalDistortion", "Perspective", "Equalize"
    ]:
        augmentation_pipeline = get_augmentations(augmentation_type)
        augmented = augmentation_pipeline(image=image)['image']
        return augmented
    elif augmentation_type == "Downsample":
        return downsample_image(image)
    elif augmentation_type == "Sharpen":
        return sharpen_image(image)
    elif augmentation_type == "TemporalShift":
        return image  # Temporal shift requires frame-level logic
    elif augmentation_type == "ProcessCroppedImage":
        return process_cropped_image(image)
    else:
        raise ValueError(f"Unknown augmentation type: {augmentation_type}")

# Process a single word folder using ThreadPoolExecutor
def process_word_folder(word_path, output_word_path, augmentation_type):
    """Process all frames in a word folder with parallel execution."""
    os.makedirs(output_word_path, exist_ok=True)
    image_files = sorted([f for f in os.listdir(word_path) if f.endswith('.png')])

    def process_frame(image_path, output_path):
        """Process a single frame."""
        image = cv2.imread(image_path)
        augmented_image = augment_image(image, augmentation_type)
        resized_image = resize_with_interpolation(augmented_image, TARGET_SIZE)
        cv2.imwrite(output_path, resized_image)

    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(process_frame, os.path.join(word_path, image_file), os.path.join(output_word_path, f"{idx:02d}.png"))
            for idx, image_file in enumerate(image_files, start=1)
        ]
        for future in as_completed(futures):
            future.result()  # Ensure all frames are processed without error

    # Print progress for this word folder
    print(f"Finished processing '{os.path.basename(word_path)}' in '{os.path.basename(output_word_path)}'. Images processed -> {len(image_files)}")

# Process a single set folder
def process_set_folder(set_path, output_set_path, augmentation_type):
    """Process all word folders in a set folder sequentially."""
    word_folders = sorted([f for f in os.listdir(set_path) if os.path.isdir(os.path.join(set_path, f))])
    for word_folder in word_folders:
        word_path = os.path.join(set_path, word_folder)
        output_word_path = os.path.join(output_set_path, word_folder)
        process_word_folder(word_path, output_word_path, augmentation_type)

# Main function
def main():
    """Main driver function."""
    set_folders = sorted([f for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))])
    augmentations = [
        "HorizontalFlip", "Brightness", "Contrast", "Rotate", "Translate", "Scale",
        "GaussianBlur", "GaussNoise", "RGBShift", "OpticalDistortion", "Perspective",
        "Equalize", "Downsample", "Sharpen", "TemporalShift", "ProcessCroppedImage"
    ]

    total_augmented_sets = 940
    source_usage = {set_name: 0 for set_name in set_folders}  # Initialize usage with all sets
    augmentation_usage = defaultdict(int)  # Track augmentation usage

    augmentation_cycle = augmentations[:]  # Create a fresh copy of augmentations
    random.shuffle(augmentation_cycle)

    for idx in range(1, total_augmented_sets + 1):
        # Ensure all augmentations are used once before repeating
        if not augmentation_cycle:
            augmentation_cycle = augmentations[:]
            random.shuffle(augmentation_cycle)

        augmentation_type = augmentation_cycle.pop()  # Get augmentation type
        augmentation_usage[augmentation_type] += 1  # Increment augmentation usage

        # Find source set with the least usage
        min_usage = min(source_usage.values())
        least_used_sets = [s for s in set_folders if source_usage[s] == min_usage]
        selected_set = random.choice(least_used_sets)
        source_usage[selected_set] += 1  # Increment usage count

        set_path = os.path.join(source_dir, selected_set)
        output_set_path = os.path.join(destination_dir, f"set_{idx:03d}")
        os.makedirs(output_set_path, exist_ok=True)

        # Process the set folder
        process_set_folder(set_path, output_set_path, augmentation_type)

        # Print progress with augmentation and usage details
        print(
            f"\nSet folder '{selected_set}' used for set_{idx:03d}. Current usage: {source_usage[selected_set]}. "
            f"Augmentation used: '{augmentation_type}' ({augmentation_usage[augmentation_type]} times so far).\n"
        )

    # Final summary
    print("\nSource Set Usage Summary:")
    for set_name, usage in sorted(source_usage.items()):
        print(f"Source set '{set_name}' used {usage} times.")

    print("\nAugmentation Usage Summary:")
    for aug_name, usage in sorted(augmentation_usage.items()):
        print(f"Augmentation '{aug_name}' applied {usage} times.")

if __name__ == "__main__":
    main()

**(optional) image processing sharpen + 5% brightness & contrast**

In [ ]:
import os
import cv2
import numpy as np
from albumentations import Compose, RandomBrightnessContrast

# Function for sharpening the image
def sharpen_image(image):
    """Apply sharpening to the image."""
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# Albumentations pipeline for brightness and contrast adjustment
def get_brightness_contrast_augmentation():
    """
    Returns a pipeline to adjust brightness and contrast without probability.
    """
    return Compose([
        RandomBrightnessContrast(
            brightness_limit=(0.05, 0.05),  # Fixed Brightness adjustment range: +5%
            contrast_limit=(0.05, 0.05),    # Fixed Contrast adjustment range: +5%
            p=1.0                         # Always apply
        )
    ])

# Unified function to apply sharpening and brightness/contrast adjustment
def process_extracted_frames(input_folder, output_folder):
    """
    Applies sharpening and brightness/contrast adjustment to all images in a folder.
    :param input_folder: Path to the folder containing images.
    :param output_folder: Path to save the processed images.
    """
    os.makedirs(output_folder, exist_ok=True)

    # Get the brightness/contrast augmentation pipeline
    augmentation_pipeline = get_brightness_contrast_augmentation()

    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith(".png"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)

            # Read the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error reading image: {input_path}")
                continue

            # Step 1: Apply sharpening
            sharpened_image = sharpen_image(image)

            # Step 2: Apply brightness and contrast adjustment
            augmented = augmentation_pipeline(image=sharpened_image)
            final_image = augmented['image']

            # Save the processed image
            cv2.imwrite(output_path, final_image)
            print(f"Processed {file_name} with sharpening and brightness/contrast adjustment.")

# Main function
def main():
    # Define input and output directories
    input_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\extracted_frames"
    output_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\final_frames_with_sharpening_brightness_contrast"

    # Apply combined processing
    print("Starting processing of images...")
    process_extracted_frames(input_folder, output_folder)
    print("Processing completed. All images are saved in the output folder.")

# Run the main function
if __name__ == "__main__":
    main()

**combined image saving and renaming**

In [4]:
def combine_images(word_path, output_set_path, word_folder_name, set_number):
    # List all frame files and sort them to maintain order
    frame_files = [f for f in os.listdir(word_path) if f.endswith('.png')]
    frame_files.sort()  # Ensure the frames are in order

    # Check that there are exactly 60 frames
    if len(frame_files) != 60:
        print(f"Warning: {word_path} does not contain exactly 60 frames. Skipping.")
        return

    # Load the first image to get dimensions
    first_image = cv2.imread(os.path.join(word_path, frame_files[0]))
    if first_image is None:
        print(f"Error: Could not read {frame_files[0]}.")
        return

    img_height, img_width, channels = first_image.shape

    # Create an empty array for the combined image (10 rows × 6 columns)
    combined_image = np.zeros((img_height * 10, img_width * 6, channels), dtype=np.uint8)

    # Place each frame into the correct position in the combined image
    for idx, frame_file in enumerate(frame_files):
        img = cv2.imread(os.path.join(word_path, frame_file))
        if img is None:
            print(f"Error: Could not read {frame_file}.")
            continue

        row = idx // 6
        col = idx % 6

        # Place the image in the combined image array
        combined_image[row * img_height:(row + 1) * img_height, col * img_width:(col + 1) * img_width] = img

    # Save the combined image with the set number and word name as the file name
    os.makedirs(output_set_path, exist_ok=True)
    combined_image_path = os.path.join(output_set_path, f'{set_number}_{word_folder_name}.png')
    cv2.imwrite(combined_image_path, combined_image)

    print(f"Combined image saved at: {combined_image_path}")

def process_all_word_folders(input_base_path, output_base_path):
    # Create the base output directory if it does not exist
    os.makedirs(output_base_path, exist_ok=True)

    # Loop through each set folder (e.g., set_01, set_02, ..., set_20)
    for set_folder in os.listdir(input_base_path):
        set_path = os.path.join(input_base_path, set_folder)
        if os.path.isdir(set_path):
            # Extract the set number from the folder name (e.g., '01' from 'set_01')
            set_number = set_folder.split('_')[-1]
            output_set_path = os.path.join(output_base_path, set_folder)
            os.makedirs(output_set_path, exist_ok=True)

            # Loop through each word folder inside the set folder
            for word_folder in os.listdir(set_path):
                word_path = os.path.join(set_path, word_folder)
                if os.path.isdir(word_path):
                    combine_images(word_path, output_set_path, word_folder, set_number)

# Define input and output base paths
input_base_path = r'C:\Users\avikd\Videos\project_videos\original_dataset_DLib_cropped_frames'
output_base_path = r'C:\Users\avikd\Videos\project_videos\original_dataset_single_images'

# Run the processing function
process_all_word_folders(input_base_path, output_base_path)

print("\nAll frames are combined and renamed")

Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_bat.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_big.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_book.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_car.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_cat.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_cup.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_dog.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_drop.png
Combined image saved at: C:\Users\avikd\Videos\project_videos\original_dataset_single_images\set_01\01_eat.png

**copying all the same word images to same word folder**

In [205]:
# Define the source directory
final_dataset1_dir = r'E:\!special\4_resized_augmented_100'
# Define the destination directory
final_dataset2_dir = r'E:\!special\5_orgnsd_augmented_100'

words = ['bat', 'cup', 'drop', 'eat', 'fish', 'hot', 'jump', 'milk', 'pen', 'red']

# Create final_datasets2 and its subfolders
os.makedirs(final_dataset2_dir, exist_ok=True)
for word in words:
    word_folder_path = os.path.join(final_dataset2_dir, word)
    os.makedirs(word_folder_path, exist_ok=True)
print(f"Created {final_dataset2_dir} with subfolders for each word.")

# Iterate through each set folder in final_datasets1 and copy images to final_datasets2
set_folders = [f for f in os.listdir(final_dataset1_dir) if os.path.isdir(os.path.join(final_dataset1_dir, f))]

for set_folder in set_folders:
    set_path = os.path.join(final_dataset1_dir, set_folder)
    for file_name in os.listdir(set_path):
        if file_name.endswith('.png'):  # Only process .png files
            # Extract the word name and prefix (e.g., '01' from '01_word1.png')
            prefix = file_name.split('_', 1)[0]  # Get the '01' portion
            base_name = file_name.split('_', 1)[-1].split('.')[0]  # Extract the base name (e.g., 'word1')
            if base_name in words:
                source_path = os.path.join(set_path, file_name)
                destination_path = os.path.join(final_dataset2_dir, base_name, file_name)
                # Copy the file to the destination
                shutil.copy2(source_path, destination_path)
                print(f"Copied {file_name} to {os.path.join(base_name, file_name)}")

                # Rename the copied file to remove the '_word' part (e.g., '01.png')
                new_file_name = f"{prefix}.png"
                new_destination_path = os.path.join(final_dataset2_dir, base_name, new_file_name)
                os.rename(destination_path, new_destination_path)
                print(f"Renamed {file_name} to {new_file_name}")

print("\nAll images have been copied and renamed.")

Created E:\!special\5_orgnsd_augmented_100 with subfolders for each word.
Copied 061_bat.png to bat\061_bat.png
Renamed 061_bat.png to 061.png
Copied 061_cup.png to cup\061_cup.png
Renamed 061_cup.png to 061.png
Copied 061_drop.png to drop\061_drop.png
Renamed 061_drop.png to 061.png
Copied 061_eat.png to eat\061_eat.png
Renamed 061_eat.png to 061.png
Copied 061_fish.png to fish\061_fish.png
Renamed 061_fish.png to 061.png
Copied 061_hot.png to hot\061_hot.png
Renamed 061_hot.png to 061.png
Copied 061_jump.png to jump\061_jump.png
Renamed 061_jump.png to 061.png
Copied 061_milk.png to milk\061_milk.png
Renamed 061_milk.png to 061.png
Copied 061_pen.png to pen\061_pen.png
Renamed 061_pen.png to 061.png
Copied 061_red.png to red\061_red.png
Renamed 061_red.png to 061.png
Copied 062_bat.png to bat\062_bat.png
Renamed 062_bat.png to 062.png
Copied 062_cup.png to cup\062_cup.png
Renamed 062_cup.png to 062.png
Copied 062_drop.png to drop\062_drop.png
Renamed 062_drop.png to 062.png
Copied 06

**resizing image to row & column: 224 x 224**

In [21]:
# Function to copy the entire directory structure and resize images
def copy_and_resize_images(source_dir, target_dir, new_size=(224, 224)):
    # Copy the directory structure
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)  # Remove if already exists to start fresh
    shutil.copytree(source_dir, target_dir)

    # Traverse the copied directory and resize all images
    for root, dirs, files in os.walk(target_dir):
        for file in files:
            if file.endswith('.png'):  # Adjust for other formats if needed
                image_path = os.path.join(root, file)
                try:
                    with Image.open(image_path) as img:
                        resized_img = img.resize(new_size, Image.LANCZOS)  # High-quality resizing
                        resized_img.save(image_path)
                        print(f"Resized and saved: {image_path}")
                except Exception as e:
                    print(f"Failed to process {image_path}: {e}")

# Paths for source and target directories
source_directory = r'D:\PycharmProjects\pro_dis_2\collected_data\6_organised_images_700'
target_directory = r'D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700'

# Run the function
copy_and_resize_images(source_directory, target_directory)

print("\nAll images have been copied and resized.")

Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\001.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\002.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\003.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\004.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\005.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\006.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\007.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\008.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\009.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\7_resized_images_700\bat\010.png
Resized and saved: D:\PycharmProjects\pr

**resizing (224 x 224) for different folder structure**

In [38]:
# Function to copy and resize images in the specified folder structure
def copy_and_resize_images(source_dir, target_dir, new_size=(224, 224)):
    """
    Copies the folder structure from source_dir to target_dir and resizes all images in the process.
    """
    # Create the target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)

    # Traverse the source directory
    for set_folder in os.listdir(source_dir):
        source_set_path = os.path.join(source_dir, set_folder)

        # Process only directories
        if os.path.isdir(source_set_path):
            target_set_path = os.path.join(target_dir, set_folder)

            # Create the corresponding directory in the target path
            os.makedirs(target_set_path, exist_ok=True)

            # Process image files within the current set folder
            for file in os.listdir(source_set_path):
                if file.endswith('.png'):
                    source_file_path = os.path.join(source_set_path, file)
                    target_file_path = os.path.join(target_set_path, file)

                    # Resize and save the image in the target directory
                    try:
                        with Image.open(source_file_path) as img:
                            resized_img = img.resize(new_size, Image.LANCZOS)  # High-quality resizing
                            resized_img.save(target_file_path)
                            print(f"Resized and saved: {target_file_path}")
                    except Exception as e:
                        print(f"Failed to process {source_file_path}: {e}")

# Define the source and target directories
source_directory = r'D:\PycharmProjects\pro_dis_2\collected_data\3_singleimg_120'
target_directory = r'D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120'

# Run the function
copy_and_resize_images(source_directory, target_directory)

print("\nAll images have been copied and resized.")

Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_bat.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_cup.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_drop.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_eat.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_fish.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_hot.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_jump.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_milk.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_pen.png
Resized and saved: D:\PycharmProjects\pro_dis_2\collected_data\4_resized_120\set_001\001_red.png
Resized and saved: D:\Pych

**(one-time) dividing total datasets into specific numbers**

In [3]:
# Total numbers to distribute
total = 40

# Step 1: Calculate the number of elements for each group based on percentage
Train_count = int(0.7 * total)  # 60% of 40
Validation_count = int(0.15 * total)  # 20% of 40
Test_count = int(0.15 * total)  # 20% of 40

# Generate the original list of numbers
original_numbers = set(range(1, total + 1))

# Print calculated counts
print(f"Distribution counts of {total}:\nA (Train): {Train_count}, B (Validation): {Validation_count}, C (Test): {Test_count}\n")

# Step 2: Generate numbers from 1 to 40 and shuffle them
numbers = list(original_numbers)
random.shuffle(numbers)

# Step 3: Distribute the numbers into A, B, and C
Train = numbers[:Train_count]  # 24 numbers for A
Validation = numbers[Train_count:Train_count + Validation_count]  # 8 numbers for B
Test = numbers[Train_count + Validation_count:Train_count + Validation_count + Test_count]  # 8 numbers for C

# Combine all distributed numbers and convert to a set
distributed_numbers = set(Train + Validation + Test)

# Find the missing number by subtracting sets
missing_number = original_numbers - distributed_numbers

# Step 4: Sort the lists in ascending order and format numbers as strings with leading zeros
Train_sorted = [f"{num:03}" for num in sorted(Train)]
Validation_sorted = [f"{num:03}" for num in sorted(Validation)]
Test_sorted = [f"{num:03}" for num in sorted(Test)]

# Combine B and C and sort
Validation_Test_combined_sorted = sorted(Validation_sorted + Test_sorted)

print("Sorted Distribution:\n")
print(f"Train ({Train_count} numbers): {Train_sorted}\n")
print(f"Validation ({Validation_count} numbers): {Validation_sorted}\n")
print(f"Test ({Test_count} numbers): {Test_sorted}\n")
print(f"Validation and Test combined ({Validation_count + Test_count} numbers): {Validation_Test_combined_sorted}\n")
print(f"The missing number is: {missing_number}")

Distribution counts of 60:
A (Train): 42, B (Validation): 9, C (Test): 9

Sorted Distribution:

Train (42 numbers): ['001', '002', '003', '004', '006', '012', '013', '014', '015', '017', '018', '019', '021', '023', '024', '026', '027', '028', '029', '030', '033', '034', '035', '036', '037', '038', '039', '040', '041', '043', '045', '046', '048', '050', '051', '052', '053', '054', '055', '056', '057', '059']

Validation (9 numbers): ['007', '008', '010', '031', '032', '042', '047', '049', '060']

Test (9 numbers): ['005', '009', '011', '016', '020', '022', '025', '044', '058']

Validation and Test combined (18 numbers): ['005', '007', '008', '009', '010', '011', '016', '020', '022', '025', '031', '032', '042', '044', '047', '049', '058', '060']

The missing number is: set()


**assigning specific images to Train, Test and Validation folders**

In [10]:
# Define the source and destination directories
source_dir = r'D:\New folder\6_organised_images_60'
destination_dir = r'D:\New folder\7_final_images_60'

# Copy word folders from source to Train, Test, and Validation subfolders in the destination
subfolders = ['Train', 'Test', 'Validation']
for subfolder in subfolders:
    subfolder_path = os.path.join(destination_dir, subfolder)
    for word_folder in os.listdir(source_dir):
        source_word_path = os.path.join(source_dir, word_folder)
        dest_word_path = os.path.join(subfolder_path, word_folder)

        if os.path.isdir(source_word_path):
            # Copy the word folder to the subfolder
            shutil.copytree(source_word_path, dest_word_path, dirs_exist_ok=True)
            print(f"Copied {word_folder} to {subfolder_path}")

print("\nAll word folders have been copied to Train, Test, and Validation.")

Copied bat to D:\New folder\7_final_images_60\Train
Copied cup to D:\New folder\7_final_images_60\Train
Copied drop to D:\New folder\7_final_images_60\Train
Copied eat to D:\New folder\7_final_images_60\Train
Copied bat to D:\New folder\7_final_images_60\Test
Copied cup to D:\New folder\7_final_images_60\Test
Copied drop to D:\New folder\7_final_images_60\Test
Copied eat to D:\New folder\7_final_images_60\Test
Copied bat to D:\New folder\7_final_images_60\Validation
Copied cup to D:\New folder\7_final_images_60\Validation
Copied drop to D:\New folder\7_final_images_60\Validation
Copied eat to D:\New folder\7_final_images_60\Validation

All word folders have been copied to Train, Test, and Validation.


In [15]:
# Define the parent directory for final_images_five\Test
final_dataset2_dir = r'D:\New folder\7_final_images_60\Train'

# List of prefixes to delete (e.g., '02.png', '03.png', etc.)
prefixes_to_delete = ['005', '007', '008', '009', '010', '011', '016', '020', '022', '025', '031', '032', '042', '044', '047', '049', '058', '060']

# Iterate through each folder in final_images_five\Test
for word_folder in os.listdir(final_dataset2_dir):
    word_folder_path = os.path.join(final_dataset2_dir, word_folder)
    if os.path.isdir(word_folder_path):
        # Iterate through files in the current word folder
        for file_name in os.listdir(word_folder_path):
            # Check if the file matches the prefix pattern
            if file_name.endswith('.png'):
                file_prefix = file_name.split('.')[0]  # Extract the prefix (e.g., '02' from '02.png')
                if file_prefix in prefixes_to_delete:
                    file_path = os.path.join(word_folder_path, file_name)
                    os.remove(file_path)
                    print(f"Deleted {file_path}")

print("\nSpecified files have been deleted in each folder.")

Deleted D:\New folder\7_final_images_60\Train\bat\005.png
Deleted D:\New folder\7_final_images_60\Train\bat\007.png
Deleted D:\New folder\7_final_images_60\Train\bat\008.png
Deleted D:\New folder\7_final_images_60\Train\bat\009.png
Deleted D:\New folder\7_final_images_60\Train\bat\010.png
Deleted D:\New folder\7_final_images_60\Train\bat\011.png
Deleted D:\New folder\7_final_images_60\Train\bat\016.png
Deleted D:\New folder\7_final_images_60\Train\bat\020.png
Deleted D:\New folder\7_final_images_60\Train\bat\022.png
Deleted D:\New folder\7_final_images_60\Train\bat\025.png
Deleted D:\New folder\7_final_images_60\Train\bat\031.png
Deleted D:\New folder\7_final_images_60\Train\bat\032.png
Deleted D:\New folder\7_final_images_60\Train\bat\042.png
Deleted D:\New folder\7_final_images_60\Train\bat\044.png
Deleted D:\New folder\7_final_images_60\Train\bat\047.png
Deleted D:\New folder\7_final_images_60\Train\bat\049.png
Deleted D:\New folder\7_final_images_60\Train\bat\058.png
Deleted D:\New